In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from itertools import combinations
import time

In [3]:
number_tickets = 10
seating_rows = 28
seating_columns = 28
number_seats = seating_rows * seating_columns
seat_depth = 0.5
seat_depth_gap = 0.2
seat_width = 0.5
seat_width_gap = 0.2

In [4]:
x_seats = np.array( [ (i % seating_columns) * (seat_width + seat_width_gap) + (seat_width / 2) for i in range(number_seats) ] )

In [5]:
y_seats = np.array( [ (i // seating_columns) * (seat_depth + seat_depth_gap) + (seat_depth / 2) for i in range(number_seats) ] )

In [6]:
#ticket_size = np.random.randint(low =1, high = 10, size = number_tickets)
ticket_size = np.ones(number_tickets)

In [7]:
ticket_size

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [8]:
def main():
    # Data
    
        
    
    # Solver
    # Create the mip solver with the CBC backend.
    solver = pywraplp.Solver.CreateSolver('assignment_mip', 'CBC')


    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if worker i is assigned to task j.
    x = {}
    for i in range(number_seats):
        x[i] = solver.IntVar(0, 1, '')

    # Constraints
    # extreme rows and columns are empty
    """for i in range(number_seats):
        if i % seating_rows == 0 or i % seating_rows == seating_rows - 1:
            solver.Add(x[i] == 0)
    
    for j in range(number_seats):
        if j < seating_columns or j > number_seats - seating_columns:
            solver.Add(x[j] == 0)"""
    
    #distance constraint
    for a, b in combinations(range(number_seats), 2):
        if (x_seats[a] - x_seats[b])**2 + (y_seats[a] - y_seats[b])**2 < (2 + seat_width + seat_width_gap) ** 2:
            solver.Add(solver.Sum([ x[a] , x[b] ]) <= 1 )
    
    
    # Objective
    objective_terms = []
    for i in range(number_seats):
        objective_terms.append(x[i])
    solver.Maximize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()
    
    #sol_matrix
    soln_matrix = np.zeros((seating_rows, seating_columns))
    # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print('Total cost = ', solver.Objective().Value(), '\n')
        for i in range(seating_rows):
            for j in range(seating_columns):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i + j * seating_columns].solution_value() > 0.5:
                    #print('Worker %d assigned to task %d.  Cost = %d' %
                          #(i, j, costs[i][j]))
                        print(i,j)
                        soln_matrix[i,j] = 1# creating and returning the solution in grid / matrix form
        return(soln_matrix)
    else:
        return status


In [9]:
t1 = time.time()
soln = main()
t2 = time.time()
t2-t1

Total cost =  56.0 

0 0
0 4
0 8
0 14
0 19
0 23
0 27
3 11
4 0
4 4
4 15
4 19
4 23
4 27
6 8
7 12
8 4
8 19
8 23
8 27
9 0
10 8
11 12
11 16
12 4
12 20
12 26
13 0
15 7
15 11
15 15
15 23
16 3
16 19
16 27
19 0
19 8
19 12
19 16
19 24
20 4
20 20
23 0
23 8
23 13
23 17
23 23
23 27
24 4
27 0
27 7
27 11
27 15
27 19
27 23
27 27


2625.521385908127

In [10]:
import numpy as np
a = np.array([[ 1.,  1.,  1.,  1.,  1.],
               [ 1.,  1.,  1.,  1.,  1.],
               [ 1.,  1.,  1.,  1.,  1.]])
np.pad(a, 1 , mode='constant')


array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 0.],
       [0., 1., 1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
soln = np.pad(soln, 1, mode='constant')

In [ ]:
fig, ax = plt.subplots(figsize=(20,15))
sns.heatmap(soln, linecolor = 'white', linewidths = 0.1, annot = True, ax = ax, cbar =False)

In [19]:
pd.DataFrame(soln).to_csv("seat_optimization_9grid25by525.csv")